Loading embeddings from Llama

In [1]:
from collections import defaultdict

import numpy as np
import json
import pickle

In [2]:
embeddings_input_path = '../data/Beauty/content_embeddings.pkl'

In [3]:
import os
print(os.path.getsize(embeddings_input_path))

446333898


In [4]:
with open(embeddings_input_path, 'rb') as f:
    data = pickle.load(f)

item_ids = np.array(data['item_id'], dtype=np.int64)
X = np.array(data['embedding'], dtype=np.float32)

Loading interactions

In [5]:
inter_json_path = "../data/Beauty/inter.json"

In [6]:
with open(inter_json_path, 'r') as f:
    user_interactions = json.load(f)

In [9]:
max_item_id = 0
for user_id_str, item_ids in user_interactions.items():
    user_id = int(user_id_str)

    if item_ids:
        max_item_id = max(max_item_id, max(item_ids))

    assert len(item_ids) >= 5, f'Core-5 dataset is used, user {user_id} has only {len(item_ids)} items'   

In [10]:
max_item_id

12100

Building cf dataset

In [14]:
from typing import Iterator, Tuple

In [22]:
def calc_pairs(history, drop_last_cnt=2) -> Iterator[Tuple[int, int]]:
    """
    in the modeling/dataset we drop last elements for leave one out
    e.g. sequence = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] (leave one out scheme, 8 - train, 9 - valid, 10 - test)
    so I decided to drop 2 last item_ids too
    """
    window_size = 2
    # we want start + window_size <= len(history), so
    # start < len(history) - window_size + 1
    start_till = len(history) - window_size + 1 - drop_last_cnt
    start_till = max(0, start_till)
    for start in range(start_till):
        window = history[start : start + window_size]
        yield window[0], window[-1]

In [23]:
def build_dataset(user_interactions):
    positive_pairs = []
    for history in user_interactions.values():
        positive_pairs.extend(calc_pairs(history))
        
    return positive_pairs    

In [24]:
pairs = build_dataset(user_interactions)

In [25]:
len(pairs)

131413

чек что в наших парах нет дубликатов

In [ ]:
cnt = 0
for (a, b) in pairs:
    if a == b:
        cnt += 1
cnt

Unknown instance spec: Please select VM configuration

In [28]:
positive_pairs_path = "../data/Beauty/positive_pairs.txt"

In [29]:
with open(positive_pairs_path, "w", encoding="utf-8") as f:
    for first, second in pairs:
        f.write(f"{first} {second}\n")